# This is for function testing functinos and outputs

In [1]:
# Imports
import sys
import tkinter as tk
from tkinter import filedialog, messagebox

from components.sql_parser import SQLConverter
from components.data_cleaner import SQLCleaner
from components.file_handler import FileHandler

In [2]:
# Initialize components
fh = FileHandler()
cleaner = SQLCleaner()
converter = SQLConverter()

In [3]:
# Ensure is_valid returns true
path = "sample_queries/Input/Tableau/1.sql"
is_valid, error = fh.validate_file(path)
is_valid

True

In [4]:
# Read raw SQL (User Input)
raw_sql = fh.read_file(path)
dict_sql = raw_sql.split("\n")
for line in dict_sql:
    print(line)


-- Customer analytics extract for Tableau → Fabric conversion test
SELECT
    [CustomerID],
    [CustomerName],
    [Region],
    [Segment],

    -- String functions
    SUBSTR([CustomerName], 1, 5) AS NameStart,
    SPLIT([Email], '@', 1) AS EmailUser,
    LENGTH([CustomerName]) AS NameLength,
    FIND([CustomerName], ' ') AS SpacePos,

    -- Boolean logic
    IF([Region] = 'West' THEN TRUE ELSE FALSE END) AS IsWest,

    -- Contains / startswith / endswith
    CONTAINS([CustomerName], 'Inc') AS HasInc,
    STARTSWITH([CustomerName], 'A') AS StartsWithA,
    ENDSWITH([CustomerName], 'LLC') AS EndsWithLLC,

    -- Date functions
    NOW() AS CurrentTS,
    TODAY() AS CurrentDate,
    DATEADD('day', -30, NOW()) AS Last30Days,
    MAKEDATE(2024, 1, 15) AS BuiltDate,
    MAKEDATETIME(2023, 12, 25, 10, 30, 00) AS BuiltDatetime,

    -- Numeric conversion
    INT([Age]) AS AgeInt,
    FLOAT([Revenue]) AS RevFloat,
    STR([Revenue]) AS RevString,
    DATE([SignupDate]) AS SignupDateCast,



In [5]:
# Use Basic Cleaning ie: remove whitespace, line breaks
cleaned = cleaner.clean_query(raw_sql)
dict_sql = cleaned.split("\n")
for line in dict_sql:
    print(line)

-- Customer analytics extract for Tableau → Fabric conversion test
SELECT
[CustomerID],
[CustomerName],
[Region],
[Segment],
-- String functions
SUBSTR([CustomerName], 1, 5) AS NameStart,
SPLIT([Email], '@', 1) AS EmailUser,
LENGTH([CustomerName]) AS NameLength,
FIND([CustomerName], ' ') AS SpacePos,
-- Boolean logic
IF([Region] = 'West' THEN TRUE ELSE FALSE END) AS IsWest,
-- Contains / startswith / endswith
CONTAINS([CustomerName], 'Inc') AS HasInc,
STARTSWITH([CustomerName], 'A') AS StartsWithA,
ENDSWITH([CustomerName], 'LLC') AS EndsWithLLC,
-- Date functions
NOW() AS CurrentTS,
TODAY() AS CurrentDate,
DATEADD('day', -30, NOW()) AS Last30Days,
MAKEDATE(2024, 1, 15) AS BuiltDate,
MAKEDATETIME(2023, 12, 25, 10, 30, 00) AS BuiltDatetime,
-- Numeric conversion
INT([Age]) AS AgeInt,
FLOAT([Revenue]) AS RevFloat,
STR([Revenue]) AS RevString,
DATE([SignupDate]) AS SignupDateCast,
-- Math
LN([Revenue]) AS LogRev,
LOG([Revenue]) AS Log10Rev,
-- Null handling
IFNULL([Profit], 0) AS ProfitNZ,

There are two testing below (on raw and on cleaned) to observe differences in comment extraction (causing issues sometimes)

In [6]:
# Extract comments (ON RAW QUERY)
no_comments_raw, comments_raw = cleaner.extract_comments(raw_sql)
dict_sql = no_comments_raw.split("\n")

# Only look at first 20 lines
for line in dict_sql[:20]:
    print(line)

for line in comments_raw[:20]:
    print(line)

SELECT
    [CustomerID],
    [CustomerName],
    [Region],
    [Segment],

    
    SUBSTR([CustomerName], 1, 5) AS NameStart,
    SPLIT([Email], '@', 1) AS EmailUser,
    LENGTH([CustomerName]) AS NameLength,
    FIND([CustomerName], ' ') AS SpacePos,

    
    IF([Region] = 'West' THEN TRUE ELSE FALSE END) AS IsWest,

    
    CONTAINS([CustomerName], 'Inc') AS HasInc,
    STARTSWITH([CustomerName], 'A') AS StartsWithA,
    ENDSWITH([CustomerName], 'LLC') AS EndsWithLLC,

-- Customer analytics extract for Tableau → Fabric conversion test
-- String functions
-- Boolean logic
-- Contains / startswith / endswith
-- Date functions
-- Numeric conversion
-- Math
-- Null handling
-- Median test (your parser should flag this)
-- LOD test (your parser must flag this)


In [7]:
# Extract comments (ON CLEANED QUERY)
no_comments_cleaned, comments_cleaned = cleaner.extract_comments(cleaned)
dict_sql = no_comments_cleaned.split("\n")

# Only look at first 20 lines
for line in dict_sql[:20]:
    print(line)
print("_____________")
for line in comments_cleaned[:20]:
    print(line)


SELECT
[CustomerID],
[CustomerName],
[Region],
[Segment],

SUBSTR([CustomerName], 1, 5) AS NameStart,
SPLIT([Email], '@', 1) AS EmailUser,
LENGTH([CustomerName]) AS NameLength,
FIND([CustomerName], ' ') AS SpacePos,

IF([Region] = 'West' THEN TRUE ELSE FALSE END) AS IsWest,

CONTAINS([CustomerName], 'Inc') AS HasInc,
STARTSWITH([CustomerName], 'A') AS StartsWithA,
ENDSWITH([CustomerName], 'LLC') AS EndsWithLLC,

NOW() AS CurrentTS,
TODAY() AS CurrentDate,
DATEADD('day', -30, NOW()) AS Last30Days,
_____________
-- Customer analytics extract for Tableau → Fabric conversion test
-- String functions
-- Boolean logic
-- Contains / startswith / endswith
-- Date functions
-- Numeric conversion
-- Math
-- Null handling
-- Median test (your parser should flag this)
-- LOD test (your parser must flag this)


In [8]:
# Split into statements (for when several statments are on the same line)
statements = cleaner.split_statements(no_comments_cleaned)
for statement in statements[:20]:
    print(statement)


SELECT
[CustomerID],
[CustomerName],
[Region],
[Segment],

SUBSTR([CustomerName], 1, 5) AS NameStart,
SPLIT([Email], '@', 1) AS EmailUser,
LENGTH([CustomerName]) AS NameLength,
FIND([CustomerName], ' ') AS SpacePos,

IF([Region] = 'West' THEN TRUE ELSE FALSE END) AS IsWest,

CONTAINS([CustomerName], 'Inc') AS HasInc,
STARTSWITH([CustomerName], 'A') AS StartsWithA,
ENDSWITH([CustomerName], 'LLC') AS EndsWithLLC,

NOW() AS CurrentTS,
TODAY() AS CurrentDate,
DATEADD('day', -30, NOW()) AS Last30Days,
MAKEDATE(2024, 1, 15) AS BuiltDate,
MAKEDATETIME(2023, 12, 25, 10, 30, 00) AS BuiltDatetime,

INT([Age]) AS AgeInt,
FLOAT([Revenue]) AS RevFloat,
STR([Revenue]) AS RevString,
DATE([SignupDate]) AS SignupDateCast,

LN([Revenue]) AS LogRev,
LOG([Revenue]) AS Log10Rev,

IFNULL([Profit], 0) AS ProfitNZ,

MEDIAN([Revenue]) AS MedianRevenue,

{ FIXED [Region] : SUM([Revenue]) } AS RegionalRevenueLOD
FROM CustomerTable
WHERE CONTAINS([Email], 'gmail')


In [ ]:
# use prepare_for_parsing to clean and validate (Basically all the steps above in one go)
try:
    prepared = cleaner.prepare_for_parsing(raw_sql)
    print(prepared.keys())

    for category, value in prepared.items():
        print("_______", category.upper(), "_______")

        if isinstance(value, str):
            for line in value.splitlines()[:20]:
                print(line)
        elif isinstance(value, list):
            for item in value[:20]:
                print(item)
        else:
            print(value)

except Exception as e:
    print(e)

# NOTE: Click view as scrollable element to see full output

dict_keys(['cleaned_query', 'comments', 'statements', 'original_query'])
_______ CLEANED_QUERY _______
SELECT
[CustomerID],
[CustomerName],
[Region],
[Segment],
SUBSTR([CustomerName], 1, 5) AS NameStart,
SPLIT([Email], '@', 1) AS EmailUser,
LENGTH([CustomerName]) AS NameLength,
FIND([CustomerName], ' ') AS SpacePos,
IF([Region] = 'West' THEN TRUE ELSE FALSE END) AS IsWest,
CONTAINS([CustomerName], 'Inc') AS HasInc,
STARTSWITH([CustomerName], 'A') AS StartsWithA,
ENDSWITH([CustomerName], 'LLC') AS EndsWithLLC,
NOW() AS CurrentTS,
TODAY() AS CurrentDate,
DATEADD('day', -30, NOW()) AS Last30Days,
MAKEDATE(2024, 1, 15) AS BuiltDate,
MAKEDATETIME(2023, 12, 25, 10, 30, 00) AS BuiltDatetime,
INT([Age]) AS AgeInt,
FLOAT([Revenue]) AS RevFloat,
_______ COMMENTS _______
-- Customer analytics extract for Tableau → Fabric conversion test
-- String functions
-- Boolean logic
-- Contains / startswith / endswith
-- Date functions
-- Numeric conversion
-- Math
-- Null handling
-- Median test (your par

In [17]:
# Run conversion (Running on comment free cleaned query)
fabric_sql, metrics, flagged = converter.convert_query(no_comments_cleaned)

split = fabric_sql.split("\n")
for line in split[:20]:
    print(line)

SELECT CustomerID,
       CustomerName,
       Region,
       SEGMENT,
       SUBSTRING(CustomerName, 1, 5) AS NameStart,
       SUBSTRING(Email, 1, CHARINDEX('@', Email) - 1) AS EmailUser,
       LEN(CustomerName) AS NameLength,
       CHARINDEX(' ', CustomerName) AS SpacePos,
       IIF(Region = 'West' THEN 1 ELSE 0 END) AS IsWest,
       CHARINDEX('Inc', CustomerName) > 0 AS HasInc,
       CHARINDEX('A', CustomerName) = 1 AS StartsWithA,
       RIGHT(CustomerName, LEN('LLC')) = 'LLC' AS EndsWithLLC,
       GETDATE() AS CurrentTS,
       CAST(GETDATE() AS DATE) AS CurrentDate,
       DATEADD('day', -30, GETDATE()) AS Last30Days,
       DATEFROMPARTS(2024, 1, 15) AS BuiltDate,
       DATETIMEFROMPARTS(2023, 12, 25, 10, 30, 00) AS BuiltDatetime,
       CAST(Age AS INT) AS AgeInt,
       CAST(Revenue AS FLOAT) AS RevFloat,
       CAST(Revenue AS VARCHAR(20)) AS RevString,


In [ ]:

# TODO: I disabled success rate, so go in and remove the code for that
for metric in metrics.to_dict().keys():
    print(metric, metrics.to_dict()[metric])

print("__________")

# TODO: Line number is not exactly correct
for item in flagged:
    print(item)

total_statements 1
successful_conversions 0
flagged_statements 2
success_rate 0.0
function_conversions {'DATE': 0, 'STRING': 0, 'AGGREGATE': 0, 'LOGICAL': 0, 'MATHEMATICAL': 0, 'OTHER': 0}
flagged_lines [(34, 'MEDIAN requires PERCENTILE_CONT(0.5) WITHIN GROUP rewrite'), (36, 'Tableau LOD expressions are not supported')]
unsupported_functions []
__________
(34, 'MEDIAN requires PERCENTILE_CONT(0.5) WITHIN GROUP rewrite')
(36, 'Tableau LOD expressions are not supported')


For conitinous support

In [33]:
"""
TODO: James I need youto add this function to the ui_controller.py file
just add a new button, and link it to this function
I would recommend to switching to a API based messaging system, so its more controllable, and can be used to configure with SendGrid (Error tracking, etc)
"""

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Email params
email_address = "jellylewis321@gmail.com"
# To get the password, 2 step verification has to be enabled, then navigate to https://myaccount.google.com/apppasswords and generate code
email_password = "vjtt vkfh cqvz rcsx"

def send_email(to_address, subject, body, sender_email, sender_password):
    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = to_address
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, to_address, msg.as_string())
        print("Email sent successfully!")

    except Exception as e:
        print("Error sending email:", e)

In [34]:
send_email(
    to_address = email_address,
    subject="Test Email",
    body="When I have _____ in my SQL, the conversion fails to recognize ______ Best Regards, and thank you",
    sender_email=email_address,
    sender_password=email_password
)


Email sent successfully!
